In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    nodes = []
    
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=1)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((4,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:33037
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:56497
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:58235
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:39375
INFO:root:('127.0.0.1', 56497) Connected to 127.0.0.1:33037
INFO:root:('127.0.0.1', 33037) Conexión aceptada con 127.0.0.1:56497
INFO:root:('127.0.0.1', 56497) Conexión aceptada con 127.0.0.1:58235
INFO:root:('127.0.0.1', 58235) Connected to 127.0.0.1:56497
INFO:root:('127.0.0.1', 33037) Connected to 127.0.0.1:58235
INFO:root:('127.0.0.1', 58235) Conexión aceptada con 127.0.0.1:33037
Processed 1 messages ({'438070627904359754': b'CONNECT_TO 127.0.0.1 58235 438070627904359754\n'})
INFO:root:('127.0.0.1', 39375) Connected to 127.0.0.1:58235
INFO:root:('127.0.0.1', 58235) Conexión aceptada con 127.0.0.1:39375
INFO:root:('127.0.0.1', 33037) Connected to 127.0.0.1:39375
INFO:root:('127.0.

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

...

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

-----------------------------SOlo se puden mandar 1, se mandan 1
-----------------------------SOlo se puden mandar 1, se mandan 1
-----------------------------SOlo se puden mandar 1, se mandan 1
-----------------------------SOlo se puden mandar 1, se mandan 1
-----------------------------SOlo se puden mandar 1, se mandan 1
-----------------------------SOlo se puden mandar 1, se mandan 1
-----------------------------SOlo se puden mandar 1, se mandan 1
-----------------------------SOlo se puden mandar 1, se mandan 1
Processed 1 messages ({'5892052709940975066': b'BEAT 5892052709940975066\n'})
Processed 1 messages ({'5892052709940975066': b'BEAT 5892052709940975066\n'})
Processed 1 messages ({'5892052709940975066': b'BEAT 5892052709940975066\n'})
Processed 1 messages ({'5962252428423591368': b'BEAT 5962252428423591368\n'})
Processed 1 messages ({'5962252428423591368': b'BEAT 5962252428423591368\n'})
Processed 1 messages ({'5962252428423591368': b'BEAT 5962252428423591368\n'})
Processed 1 

In [1]:
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
import threading

nodes = []
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(1)

    # Check if they are connected
    for node in nodes:
        print(len(node.neightboors))
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    time.sleep(0.3)
    nodes[1].stop()
    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()


nodes = []
for _ in range(6):
    test_node_down_on_learning(4)
    break


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:41233
aaaaaaa 7.152557373046875e-07
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:51209
aaaaaaa 7.152557373046875e-07
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:34731
aaaaaaa 7.152557373046875e-07
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:51613
aaaaaaa 4.76837158203125e-07
INFO:root:('127.0.0.1', 51209) Connected to 127.0.0.1:41233
INFO:root:('127.0.0.1', 41233) Conexión aceptada con 127.0.0.1:51209
aaaaaaa 6.67572021484375e-06
aaaaaaa 5.245208740234375e-06
aaaaaaa 5.245208740234375e-06
aaaaaaa 5.0067901611328125e-06
INFO:root:('127.0.0.1', 34731) Connected to 127.0.0.1:51209
INFO:root:('127.0.0.1', 51209) Conexión aceptada con 127.0.0.1:34731
INFO:root:('127.0.0.1', 41233) Connected to 127.0.0.1:34731
Processed 1 messages ({'-2127817028240986534': b'CONNECT_TO 127.0.0.1 34731 -2127817028240986534\n'})
INFO:root:('127.0.0.1', 34731) Co

INFO:root:(('127.0.0.1', 41233)) Waiting other node votes.
INFO:root:(('127.0.0.1', 51209)) Waiting other node votes.
INFO:root:('127.0.0.1', 34731) Train set of 4 nodes. [('127.0.0.1', 51209), ('127.0.0.1', 41233), ('127.0.0.1', 34731), ('127.0.0.1', 51613)]
INFO:root:('127.0.0.1', 51613) Train set of 4 nodes. [('127.0.0.1', 51209), ('127.0.0.1', 41233), ('127.0.0.1', 34731), ('127.0.0.1', 51613)]
INFO:root:('127.0.0.1', 41233) Train set of 4 nodes. [('127.0.0.1', 51209), ('127.0.0.1', 41233), ('127.0.0.1', 34731), ('127.0.0.1', 51613)]
INFO:root:('127.0.0.1', 51209) Train set of 4 nodes. [('127.0.0.1', 51209), ('127.0.0.1', 41233), ('127.0.0.1', 34731), ('127.0.0.1', 51613)]
INFO:root:(('127.0.0.1', 34731)) Evaluating...
INFO:root:(('127.0.0.1', 51613)) Evaluating...
INFO:root:(('127.0.0.1', 41233)) Evaluating...
INFO:root:(('127.0.0.1', 51209)) Evaluating...
INFO:root:(('127.0.0.1', 34731)) Training...
INFO:root:(('127.0.0.1', 51613)) Training...
INFO:root:(('127.0.0.1', 41233)) Tra

In [1]:
a = {('127.0.0.1', 35159): 185, ('127.0.0.1', 54181): 62}
a.items()

dict_items([(('127.0.0.1', 35159), 185), (('127.0.0.1', 54181), 62)])

In [1]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_connect_to_train_running(four_nodes):
    n1,n2,n3,n4 = four_nodes

    # Connect Nodes (unless the n4)
    n1.connect_to(n2.host,n2.port)
    n3.connect_to(n1.host,n1.port)
    time.sleep(0.1) #Esperar por la asincronía

    n1.set_start_learning(2,1)

    time.sleep(2)
    
    n4.connect_to(n1.host,n1.port,full=False)

    time.sleep(2)

    #assert n4.round is not None

    # If the new node doesent know that learning is running, it will cause inanition in the other nodes (that are waiting for him to finish)

    while all([n1.round is not None for n in [n1,n2,n3,n4]]):
        time.sleep(0.1)

    for n in [n1,n2,n3,n4]:
        n.stop()
        time.sleep(0.1)
        
        
nodes = []
    
for i in range(4):
    node = Node(MLP(),MnistFederatedDM())
    node.start()
    nodes.append(node)
    
test_connect_to_train_running(nodes)


Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:38537
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:42513
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:41845
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo listening at 127.0.0.1:60271
INFO:root:('127.0.0.1', 38537) Connected to 127.0.0.1:42513
INFO:root:('127.0.0.1', 42513) Conexión aceptada con 127.0.0.1:38537
INFO:root:('127.0.0.1', 41845) Connected to 127.0.0.1:38537
INFO:root:('127.0.0.1', 38537) Conexión aceptada con 127.0.0.1:41845
INFO:root:('127.0.0.1', 42513) Connected to 127.0.0.1:41845
INFO:root:('127.0.0.1', 41845) Conexión aceptada con 127.0.0.1:42513
INFO:root:(('127.0.0.1', 38537)) Broadcasting start learning...
INFO:root:(('127.0.0.1', 38537)) Sending Initial Model Weights
INFO:root:(('127.0.0.1', 38537)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 42513)) Broadcasting Number of Samples...
INFO:root:(('127.0.0.1',

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:('127.0.0.1', 38537) Conexión aceptada con 127.0.0.1:60271
INFO:root:('127.0.0.1', 60271) Connected to 127.0.0.1:38537
TO IMPLEMET WHEN THE TOPOLOGY WAS NOT FULLY CONNECTED
DEBUG:root:Closed connection: ('127.0.0.1', 60271)
DEBUG:root:Closed connection: ('127.0.0.1', 38537)
INFO:root:(('127.0.0.1', 38537)) Evaluated. Losss: 2.301456928253174, Metric: 0.11810000240802765. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 38537)) Training...
INFO:root:(('127.0.0.1', 42513)) Evaluated. Losss: 2.301456928253174, Metric: 0.11810000240802765. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 42513)) Training...


...

INFO:root:(127.0.0.1:38537) Model added (1/3) from ('127.0.0.1', 38537)
INFO:root:(127.0.0.1:42513) Model added (1/3) from ('127.0.0.1', 42513)
INFO:root:(('127.0.0.1', 38537)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 42513)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(127.0.0.1:41845) Model added (1/3) from ('127.0.0.1', 42513)
INFO:root:(127.0.0.1:38537) Model added (2/3) from ('127.0.0.1', 42513)
INFO:root:(127.0.0.1:41845) Model added (2/3) from ('127.0.0.1', 38537)
INFO:root:(127.0.0.1:42513) Model added (2/3) from ('127.0.0.1', 38537)
INFO:root:(127.0.0.1:41845) Model added (3/3) from ('127.0.0.1', 41845)
INFO:root:(127.0.0.1:41845) Agregating models.
INFO:root:(('127.0.0.1', 41845)) Broadcasting model to 2 clients. (size: 548864 bytes)
INFO:root:(('127.0.0.1', 41845)) Waiting other nodes.
INFO:root:(127.0.0.1:38537) Model added (3/3) from ('127.0.0.1', 41845)
INFO:root:(127.0.0.1:42513) Model added (3/3) from ('127.0.0.1

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

INFO:root:(('127.0.0.1', 42513)) Evaluated. Losss: 0.07695037126541138, Metric: 0.9758999943733215. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 42513)) Finish!!.
INFO:root:(('127.0.0.1', 41845)) Evaluated. Losss: 0.07695037126541138, Metric: 0.9758999943733215. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 38537)) Evaluated. Losss: 0.07695037126541138, Metric: 0.9758999943733215. (Check tensorboard for more info)
INFO:root:(('127.0.0.1', 41845)) Finish!!.
INFO:root:(('127.0.0.1', 38537)) Finish!!.
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 38537 y desconectándose de 2 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 38537)
DEBUG:root:Closed connection: ('127.0.0.1', 38537)
DEBUG:root:Closed connection: ('127.0.0.1', 42513)
DEBUG:root:Closed connection: ('127.0.0.1', 41845)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 42513 y desconectándose de 1 nodos
DEBUG:root:Closed connection: ('127.0.0.1', 42513)
DEBUG:root:Closed connec

In [4]:
hash("msg" + str(datetime.now()))

3116227613539867172

In [3]:
from datetime import datetime

now = datetime.now()
str(now)

'2022-06-27 17:13:54.992782'

In [6]:
hash("caca".encode("utf-8"))

5682035749826620771

In [8]:
str(time.time())

'1656343107.3914704'

In [3]:
"-1".isdecimal()

False

In [4]:
a = [(1,1)]
b = {1:1}

a = a + list(b)
print(a)

[(1, 1), 1]


In [1]:
None in []

False

In [9]:
a = {1:2} | {1:1}
print(a)

{1: 1}
Processed {'-8414875811797426002': b'BEAT -8414875811797426002\n'} messages from <NodeConnection(node_connection-127.0.0.1:35411-127.0.0.1:47939, started 140116999796480)>
Processed {'7452549549026162342': b'BEAT 7452549549026162342\n'} messages from <NodeConnection(node_connection-127.0.0.1:47939-127.0.0.1:35411, started 140117347022592)>
Processed {'8384566461848987428': b'BEAT 8384566461848987428\n'} messages from <NodeConnection(node_connection-127.0.0.1:35411-127.0.0.1:47939, started 140116999796480)>
Processed {'-1284154675020740598': b'BEAT -1284154675020740598\n'} messages from <NodeConnection(node_connection-127.0.0.1:47939-127.0.0.1:35411, started 140117347022592)>
Processed {'9016479497930719462': b'BEAT 9016479497930719462\n'} messages from <NodeConnection(node_connection-127.0.0.1:35411-127.0.0.1:47939, started 140116999796480)>
Processed {'5086953804389111734': b'BEAT 5086953804389111734\n'} messages from <NodeConnection(node_connection-127.0.0.1:47939-127.0.0.1:35

In [3]:
a = {'-6292952804535679250': b'NUM_SAMPLES 54000 10000 -6292952804535679250\n', '8763277242718051430': b'NUM_SAMPLES 54000 10000 8763277242718051430\n'}
a

{'-6292952804535679250': b'NUM_SAMPLES 54000 10000 -6292952804535679250\n',
 '8763277242718051430': b'NUM_SAMPLES 54000 10000 8763277242718051430\n'}

In [4]:
[k for k,v in a]

ValueError: too many values to unpack (expected 2)

In [6]:
list(a.values())

[b'NUM_SAMPLES 54000 10000 -6292952804535679250\n',
 b'NUM_SAMPLES 54000 10000 8763277242718051430\n']

In [2]:
a = "NUM_SAMPLES 54000 10000 7567134918904634235\nVOTE_TRAIN_SET  127.0.0.1 35157 776 127.0.0.1 35157 384 127.0.0.1 35157 249 \\VOTE_TRAIN_SET 4066132568365956543\n"

a.split()

['NUM_SAMPLES',
 '54000',
 '10000',
 '7567134918904634235',
 'VOTE_TRAIN_SET',
 '127.0.0.1',
 '35157',
 '776',
 '127.0.0.1',
 '35157',
 '384',
 '127.0.0.1',
 '35157',
 '249',
 '\\VOTE_TRAIN_SET',
 '4066132568365956543']

In [4]:
a = {1:5}

for x in a.items():
    print(x)

(1, 5)


In [1]:
from p2pfl.base_node import BaseNode
import time

def test_node_paring():
    n1 = BaseNode()
    n2 = BaseNode()
    n1.start()
    n2.start()
    # Conexión
    n1.connect_to(n2.host,n2.port)
    time.sleep(0.1) #Esperar por la asincronía
    assert len(n1.neightboors) == len(n2.neightboors)==1

    # Desconexión
    n2.neightboors[0].stop()
    time.sleep(0.1) #Esperar por la asincronía
    assert len(n1.neightboors) == len(n2.neightboors)== 0
    
    n1.stop()
    n2.stop()

test_node_paring()

INFO:root:Nodo listening at 127.0.0.1:50711
INFO:root:Nodo listening at 127.0.0.1:46033
INFO:root:('127.0.0.1', 50711) Connected to 127.0.0.1:46033
INFO:root:('127.0.0.1', 46033) Conexión aceptada con 127.0.0.1:50711
DEBUG:root:Closed connection: ('127.0.0.1', 46033)
DEBUG:root:Closed connection: ('127.0.0.1', 50711)
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 46033 y desconectándose de 0 nodos
INFO:root:Bajando el nodo, dejando de escuchar en 127.0.0.1 50711 y desconectándose de 0 nodos


In [1]:
[] - []

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [3]:
set([])[0]

TypeError: 'set' object is not subscriptable